In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import json
import os

In [2]:
all_data=pd.read_csv('/eslap/legacy/mnist.csv')
train,test=train_test_split(all_data,test_size=0.25)
del all_data 
batch_size=128
num_batches=int(train.shape[0]/batch_size)

In [3]:
def to_one_hot(targets,num_classes):
    aux = np.zeros((targets.shape[0], num_classes))
    aux[np.arange(targets.shape[0]), targets] = 1
    return aux

In [4]:
Y_train = to_one_hot(train['label'].values, num_classes = 10)
Y_test = to_one_hot(test['label'].values, num_classes = 10)
X_train = train.drop(labels = ['label'],axis = 1).values/255.0
X_test= test.drop(labels = ['label'],axis = 1).values/255.0
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((31500, 784), (31500, 10), (10500, 784), (10500, 10))

In [16]:
# Set parameters
learning_rate = 0.001
training_iteration = 30
batch_size = 128
display_step = 2

# TF graph input
x = tf.placeholder(tf.float64, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float64, [None, 10]) # 0-9 digits recognition => 10 classes

W_1 = tf.Variable(tf.truncated_normal([784, 64],dtype=tf.float64))
b_1 = tf.Variable(tf.zeros([64],dtype=tf.float64))
    
W_2 = tf.Variable(tf.truncated_normal([64, 10],dtype=tf.float64))
b_2 = tf.Variable(tf.zeros([10],dtype=tf.float64))


# Construct a linear model
h_1=tf.nn.relu(tf.matmul(x, W_1) + b_1)
h_2=tf.matmul(h_1, W_2) + b_2
model = tf.nn.softmax(h_2)  # Softmax

loss = -tf.reduce_sum(y*tf.log(model))

predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(predictions, "float"))

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

init = tf.global_variables_initializer()


In [19]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    epsilon=0.9
    test_acc = 0.0
    iteration=0
    while test_acc < epsilon and iteration < 30:
        avg_acc = 0.0
        avg_cost = 0.0
        for i in range(num_batches):
            lower_index=batch_size*i
            if i==num_batches-1:
                batch = X_train[lower_index:]
                batch_labels=Y_train[lower_index:]
            else:
                upper_index=batch_size*(i+1)
                batch = X_train[lower_index:upper_index]
                batch_labels=Y_train[lower_index:upper_index]
            
            sess.run(train_step, feed_dict={x: batch, y: batch_labels})
            
            #print(sess.run(W_1))
            avg_cost += sess.run(loss, feed_dict={x: batch, y: batch_labels})/float(num_batches)
            avg_acc += sess.run(accuracy, feed_dict={x: batch, y: batch_labels})/float(num_batches)
            
        test_loss = sess.run(loss, feed_dict={x: X_test, y: Y_test})
        test_acc = sess.run(accuracy, feed_dict={x: X_test, y: Y_test})
        
        print ("Iteration:%d"%iteration, "training loss= %f"%avg_cost, "training accuracy= %f"%avg_acc,\
               "test accuracy= %f"%test_acc)
        iteration+=1
        
    print("Training completed!")


Iteration:0 training loss= 350.058426 training accuracy= 0.760246 test accuracy= 0.800571
Iteration:1 training loss= 98.898536 training accuracy= 0.848901 test accuracy= 0.824000
Iteration:2 training loss= 70.398827 training accuracy= 0.868419 test accuracy= 0.839048
Iteration:3 training loss= 58.200318 training accuracy= 0.880836 test accuracy= 0.848381
Iteration:4 training loss= 51.146387 training accuracy= 0.892041 test accuracy= 0.857238
Iteration:5 training loss= 46.342218 training accuracy= 0.899886 test accuracy= 0.863619
Iteration:6 training loss= 42.767478 training accuracy= 0.906457 test accuracy= 0.868857
Iteration:7 training loss= 40.014080 training accuracy= 0.912110 test accuracy= 0.873524
Iteration:8 training loss= 37.752347 training accuracy= 0.916874 test accuracy= 0.877905
Iteration:9 training loss= 35.823030 training accuracy= 0.920939 test accuracy= 0.882286
Iteration:10 training loss= 34.155328 training accuracy= 0.925226 test accuracy= 0.885429
Iteration:11 traini